In [1]:
import pandas as pd
import numpy as np
import time
import os

t = time.localtime()
timestamp = time.strftime('%y%m%d-%H%M', t)

# read excel
df = pd.read_excel('H://Current_Working_Folder//Reports Analytics//201909//201909-t_worc_families-20191108_1418.xlsx')

# assign path and reporitng month
drive = 'H://Current_Working_Folder//Reports Analytics//'
reporting_month = str(df.loc[0,'reporting_month'])

# Filter with column names ending with amt, amount, nbr_month
df_amt = df.filter(regex = 'amt$|amount$|amount$|nbr_month$|nbr_months', axis='columns')

# Describe the filtered dataframe
df_des = df_amt.describe()

# Write the describe to excel
writer = pd.ExcelWriter(drive + reporting_month + '//' + reporting_month + '_' + 'report' + '-' + timestamp + '.xlsx')
#df_des.to_excel(writer, index = False)
df_des.to_excel(writer, sheet_name='family_stat')
writer.save()

In [2]:
df.columns

Index(['id', 'category', 'reporting_month', 'case_number', 'county_fips_code',
       'stratum', 'zip_code', 'funding_stream', 'disposition', 'new_applicant',
       'family_size', 'family_type', 'subsid_housing', 'medical_assist',
       'food_stamps', 'food_stamps_amt', 'subsid_child_care',
       'subsid_child_care_amt', 'child_support_amt', 'cash_resource_amt',
       'cash & cash equivalents_amount', 'cash & cash equivalents_nbr_month',
       'tanf child care_amount', 'tanf child care_children_covered',
       'tanf child care_nbr_months', 'transportation_amount',
       'transportation_nbr_months', 'transitional services_amount',
       'transitional services_nbr_months', 'other_amount', 'other_nbr_months',
       'reason for amount of reduction_sanction reduction_amount',
       'reason for amount of reduction_work requirements sanction',
       'reason for amount of reduction_Family Sanction for an Adult with No High School Diploma or Equivalent',
       'reason for amount of 

In [ ]:
df.shape

In [3]:
df['subsid_child_care'].value_counts()

No subsidized child care received    2358
Name: subsid_child_care, dtype: int64

In [4]:
df['subsid_child_care_amt'].sum()

0

In [5]:
df['tanf child care_children_covered'].value_counts()

0    2358
Name: tanf child care_children_covered, dtype: int64

In [6]:
df['tanf child care_amount'].sum()

0

In [7]:
cat = df.drop(df_amt, axis='columns')
#cat1 = cat.drop((cat.loc[:,'reporting_month':'stratum']),axis='columns')
cat1 = cat.drop(['id','reporting_month','case_number','county_fips_code','stratum','disposition','worker_id',
                 'case_manager','area','created_by','updated_by','created_at','updated_at','ans_county',
                 'local_office','waiver_grps'], axis='columns')
cat2 = cat1.drop(cat1.filter(regex = '^reason|^other'), axis='columns')

In [8]:
reason_cat = cat1.filter(regex = '^reason', axis='columns')

for c in reason_cat.columns:
    print(reason_cat[c].value_counts())
    print('---')


No     2166
Yes     192
Name: reason for amount of reduction_work requirements sanction, dtype: int64
---
No    2358
Name: reason for amount of reduction_Family Sanction for an Adult with No High School Diploma or Equivalent, dtype: int64
---
No    2358
Name: reason for amount of reduction_Sanction for Teen Parent not Attending School, dtype: int64
---
No     2255
Yes     103
Name: reason for amount of reduction_Non-Cooperation with Child Support, dtype: int64
---
No     2357
Yes       1
Name: reason for amount of reduction_Failure to Comply with an Individual Responsibility Plan, dtype: int64
---
No     2355
Yes       3
Name: reason for amount of reduction_Other Sanction, dtype: int64
---
0    2358
Name: reason for amount of reduction_Recoupment of Prior Overpayment, dtype: int64
---


In [9]:
#((df.filter(regex = '^reason'))).head(20)

In [10]:
for c in cat2.columns:
    print((cat2[c].value_counts().to_frame('count').rename_axis(c)).reset_index())
    print('---')

   category  count
0       Tea   2103
1  WorkPays    255
---
     zip_code  count
0       71854     74
1       72301     69
2       72209     67
3       72401     63
4       72390     53
5       72076     49
6       72204     48
7       72143     47
8       72335     45
9       71601     41
10      72114     37
11      72315     37
12      72764     35
13      72117     33
14      71603     32
15      72956     32
16      72901     31
17      72206     31
18      72342     31
19      72450     31
20      72118     30
21      72904     30
22      72032     27
23      72601     26
24      72360     25
25      71730     23
26      72701     22
27      72501     21
28      71913     21
29      71953     20
..        ...    ...
332     72072      1
333     72415      1
334     72413      1
335     72080      1
336     72954      1
337     72383      1
338     71658      1
339     72102      1
340     72106      1
341     72732      1
342     72469      1
343     72024      1
344     72940  

In [ ]:
worc_fam = df
fam_type = worc_fam.groupby(['category','family_type']).agg({'case_number' : 'nunique'}).rename({'case_number': 'case_count'}, axis='columns')#.reset_index()
fam_type